In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import numpy as np
import seaborn as sns
from torch import nn
import sys
sys.path.append('../')
from fun.models import *
sns.set_style("whitegrid")
from torch.utils.data import random_split
import pandas as pd

In [ ]:

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])


dataset = torchvision.datasets.CIFAR10(
    root="../data",
    train=True,
    download=True,
    transform=transform  
)


torch.manual_seed(42)
valid_size = int(0.1 * len(dataset))
train_size = len(dataset) - valid_size


train_ds, valid_ds = random_split(dataset, [train_size, valid_size])
len(train_ds), len(valid_ds)
    

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

def train(dataloader, model, loss_fn, optimizer):

    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.train()
    train_loss, correct = 0, 0
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 ==0:
            print(batch)

        train_loss += loss.item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()


    train_loss /= num_batches
    correct /= size
    print(f" Train Accuracy: {(100*correct):>0.1f}%, Train Avg loss {train_loss:>8f} \n")
    return correct, train_loss


def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size

    print(f"Test Accuracy: {(100*correct):>0.1f}%, Test Avg loss: {test_loss:>8f} \n")
    return correct, test_loss
    

In [ ]:
epochs = 30
learning_rate = 1e-3

batch_size = 8
step_size = 7
gamma = 0.5
weight_decay=0.1


data = np.array([[0]*4]*epochs)
adam_wd_schedul_history = pd.DataFrame(data, columns = ["test_acc", "train_acc", "test_loss", "train_loss"])


train_dataloader = DataLoader(
        train_ds,
        batch_size=batch_size, 
        shuffle=True
        )
test_dataloader = DataLoader(
    valid_ds, 
    batch_size=batch_size,
    shuffle=False
    )

#Freeze last layers (layer4)
net = Resnet18_2().to(device)
for param in net.conv_layers_frozen.parameters():    
    param.requires_grad = False


criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(net.parameters(), lr=learning_rate, weight_decay=weight_decay)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

for t in range(epochs):
    
    print(f"Epoch {t+1}\n-------------------------------")

    train_acc, train_loss = train(train_dataloader, net, criterion, optimizer)
    test_acc, test_loss = test(test_dataloader, net, criterion)
    lr_scheduler.step()
    adam_wd_schedul_history.loc[t,"train_acc"] = train_acc        
    adam_wd_schedul_history.loc[t,"test_acc"] = test_acc
    adam_wd_schedul_history.loc[t,"train_loss"] = train_loss
    adam_wd_schedul_history.loc[t,"test_loss"] = test_loss

# adam_wd_schedul_history.to_csv("adam_wd_schedul_history.csv")
# !cp adam_wd_schedul_history.csv "drive/My Drive/DL/Resnet18_2/adam/"

 
    